In [ ]:
#위도 경도가 return 형식이 조금씩 달라서 map 이 이상하게 생성될 수 있으므로
#결과가 이상하면 X,Y값의 위치를 서로 바꾸어 보아보자.

In [1]:
import requests 
import ast
import re
import folium
import numpy as np
import pandas as pd
import configparser
import googlemaps

In [2]:
class RoadFind:
    def __init__(self):
        self.url = "https://api2.sktelecom.com/tmap/routes?version=1&format=html"
        self.headers = {}
        self.data = {}
        
        self.road_list = []
        self.made_map = self.map_make((37.449627,126.653116))
        self.total_distance = 0
        self.total_time = 0
        
        
        self.google_key = ''
        self.gmaps=[]
        self.set_up()
    def set_up(self):
        config = configparser.ConfigParser()
        config.read('config.ini')
        apikey = config['api_key']["api_key"]
        self.google_key = config['google_key']['api_key']
        self.headers['appKey'] = apikey
        self.data={
            "startX" : "",
            "startY" : "",
            "endX" : "",
            "endY" : "",
            "reqCoordType" : "WGS84GEO",
            "resCoordType" : "WGS84GEO",
            "angle" : "172",
            "searchOption" : 10
        }
        self.gmaps= googlemaps.Client(key=self.google_key)
    '''
    google
    '''    
    def geocode_make(self,place):
        '''
        input place : string
        output geocode : int [lng, lat]
        '''
        result = self.gmaps.geocode(place)
        lat = result[0]['geometry']['location']['lat']
        lng = result[0]['geometry']['location']['lng']
        return [lng,lat]
    
    
    def find_load(self,start_place,end_place):
        start_geo = self.geocode_make(start_place)
        end_geo = self.geocode_make(end_place)
        self.set_start_end_points(start_geo[0],start_geo[1],\
                             end_geo[0],end_geo[1])
        res = self.post()
        self.get_road_info(res)
        self.map_make((start_geo[1],start_geo[0]))
        self.join_load(self.made_map,self.road_list)
        result = self.map_show()
        return result
    '''
    tmap
    '''    
    def set_start_end_points(self,startX,startY,endX,endY):
        self.data['startX'] = str(startX)
        self.data['startY'] = str(startY)
        self.data['endX'] = str(endX)
        self.data['endY'] = str(endY)
        
    def post(self):
        res = requests.post(self.url,headers=self.headers,data=self.data)
        if(res.status_code==200):
            res = ast.literal_eval(res.text)
            return res
        else :
            return res.status_code
    
    def get_road_info(self,res):
        self.total_distance = res['features'][0]['properties']['totalDistance']
        self.total_time = res['features'][0]['properties']['totalTime']
        self.road_list = []
        for i in range(len(res['features'])):
            for j in range(len(res['features'][i]['geometry']['coordinates'])):
                if isinstance(res['features'][i]['geometry']['coordinates'][j],list):
                    temp=res['features'][i]['geometry']['coordinates'][j]
                    x = temp[1]
                    y = temp[0]
                    self.road_list.append((x,y))
                    
    def map_make(self,location=[], zoom=15):
        self.made_map = folium.Map(location = location, zoom_start= zoom) #location
        return self.made_map
    
    def join_load(self,map_name,road_list):
        res = folium.PolyLine(road_list).add_to(map_name)
        return res 
    
    def map_show(self,zoom=15):
        return self.made_map
    def total_distance_(self):
        return str(self.total_distance)+"m"
    def total_time_(self):
        return str(round(self.total_time/60,3))+"분"

In [3]:
rf = RoadFind()
rf.map_make([36.67965641069,126.654492042609])

In [43]:
rf_2 = RoadFind()
rf_2.find_load('인하대 후문', '주안역삼거리')

In [45]:
rf_2.total_time_()

'10.483분'

In [25]:
"""
라인 이것저것 추가하려면 이렇게이용해야함
rf = RoadFind()
rf.set_start_end_points(126.653116,37.449627,126.6803348,37.4639865)
res = rf.post()
rf.get_road_info(res)
rf.join_load(rf.made_map,rf.road_list)
rf.map_show()

"""

'15.5분'